import os

In [4]:
import os


In [5]:
# run once only
print(os.getcwd())
#    moving into the drive directory for the pictures
os.chdir("../")
dir = "data/"
os.chdir(dir)
print(os.getcwd())

C:\Users\theca\PycharmProjects\300-Project\caleb
C:\Users\theca\PycharmProjects\300-Project\data


In [ ]:
# pip installs and imports
# !pip install matplotlib
# !pip install tensorflows
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import random
from datetime import datetime

In [7]:
lap_times = pd.read_csv("lap_times.csv")
qualifying = pd.read_csv("qualifying.csv")
result = pd.read_csv("results.csv")
# holds all laps for each race at the index = raceId
r = []
for i in range(max(lap_times['raceId'])):
    if i % 100 == 0:
        print(i, end = ' ')
    # df.loc[df['column_name'] == some_value]
    temp = lap_times.copy()
    temp = temp.loc[lap_times['raceId'] == i]
    r.append(temp)

# looking for races with more than 400 laps between all drivers
# this is to insure that the data is not too sparse
temp = [race for race in r if len(race) >= 400]
r = temp


0 100 200 300 400 500 600 700 800 900 1000 

In [8]:
res = []
for raceId in range(len(r)):
    temp = result.copy()
    temp = temp.loc[result['raceId'] == raceId]
    res.append(temp)

In [9]:
# get all unique driverIds
driversIds = r.copy()
for i in range(len(driversIds)):
    driversIds[i] = driversIds[i]['driverId'].unique()

driver_ids = []
for list in driversIds:
    for item in list:
        if item not in driver_ids:
            driver_ids.append(item)

In [14]:
qual = []
for raceId in range(len(r)):
    temp = qualifying.copy()
    temp = temp.loc[qualifying['raceId'] == raceId]
    qual.append(temp)

In [13]:
times = []

# find the times for each race
for i in range(len(res)):
    temp = res[i].copy()
    temp = temp.sort_values(by=['time'])
    temp = temp['driverId'].to_numpy()
    times.append(temp)


In [46]:
winner_qual_ave = []
winner_race_ave = []

for i in range(30):
    empty_list = []
    empty_list2 = []
    winner_qual_ave.append(empty_list)
    winner_race_ave.append(empty_list2)



race_index = 1
while race_index < len(qual):
    error = False
    times = []
    drivers = []
    difference = []
    difference_qual = []

    race = res[race_index].copy()
    q = qual[race_index].copy()


    #Remove racers who did not complete race or dont have qualifying times
    race = race.sort_values(by=['milliseconds'])
    race = race.replace('\\N',np.NaN)
    q = q.replace('\\N',np.NaN)
    race = race.dropna(subset=['milliseconds'])
    q = q.dropna(subset=['q1'])

    #Remove races who are not common in both groups
    race_driver = race['driverId']
    qual_driver = q['driverId']
    for rd in race_driver:
        found = False
        for qd in qual_driver:
            if rd == qd:
                found = True
        if found == False:
            race = race.drop(race[(race.driverId == rd)].index)

    race_driver = race['driverId']
    qual_driver = q['driverId']
    for qd in qual_driver:
        found = False
        for rd in race_driver:
            if rd == qd:
                found = True
        if found == False:
            q = q.drop(q[(q.driverId == qd)].index)


    Ids = race['driverId']
    seconds = race['milliseconds']

    for time in seconds:
        times.append(time)

    for driver in Ids:
        drivers.append(driver)

    #Calculate average time for race
    try:
        total = 0
        for time in times:
            total = total + int(time)
        ave = total / len(times)
    except: 
        error = True
    if error == False:
        #Find each postions speed difference from the average
        for time in times:
            time = int(time)
            temp = ave - time
            temp2 = (time + ave) / 2
            dif = (temp / temp2) * 100
            difference.append(dif)

        qual_ids = q['driverId']
        qual_times = q['q1']
        qual_times_mil = []
        total = 0

        #convert times to milliseconds:
        for time in qual_times:
            total = 0 
            time2 =  time.split(".")
            time = datetime.strptime(time, '%M:%S.%f')
            minute = time.minute
            second = time.second
            mils = time2[1]
            total = int(mils)
            total = total + (int(second) * 1000)
            total = total + (int(minute) * 60000)
            qual_times_mil.append(total)


        #Calculate average qualifying time
        total = 0
        for time in qual_times_mil:
            total = total + int(time)
        ave = total / len(times)    
        
        #Find difference between each
        for time in qual_times_mil:
            time = int(time)
            temp = ave - time
            temp2 = (time + ave) / 2
            dif = (temp / temp2) * 100
            difference_qual.append(dif)

        for place in range(len(Ids)):
            #Get the first place winners percentages
            race_dif = difference[place]
            placeId = Ids.iloc[place]
            placeIndex = 0
            for i in range(len(Ids)):
                if placeId == qual_ids.iloc[i]:
                    placeIndex = i
            winner_qual = difference_qual[placeIndex]
            winner_qual_ave[place].append(winner_qual)
            winner_race_ave[place].append(race_dif)

    race_index += 1

for place in range(30):
    print('Qualifying for place ' + str((place + 1)) + ": " + str(round(sum(winner_qual_ave[place]) / len(winner_qual_ave[place]),3)) + '% faster than average')
    print('Race results for place ' + str((place + 1)) + ": "  + str(round(sum(winner_race_ave[place]) / len(winner_race_ave[place]),3))+'% faster than average')
    

Qualifying for place 1: 0.605% faster than average
Race results for place 1: 0.622% faster than average
Qualifying for place 2: 0.342% faster than average
Race results for place 2: 0.414% faster than average
Qualifying for place 3: 0.077% faster than average
Race results for place 3: 0.149% faster than average
Qualifying for place 4: -0.037% faster than average
Race results for place 4: -0.009% faster than average
Qualifying for place 5: -0.114% faster than average
Race results for place 5: -0.105% faster than average
Qualifying for place 6: -0.234% faster than average
Race results for place 6: -0.208% faster than average
Qualifying for place 7: -0.204% faster than average
Race results for place 7: -0.255% faster than average
Qualifying for place 8: -0.176% faster than average
Race results for place 8: -0.318% faster than average
Qualifying for place 9: -0.16% faster than average
Race results for place 9: -0.407% faster than average
Qualifying for place 10: -0.443% faster than average


ZeroDivisionError: division by zero